In [17]:
# Dependencies
import numpy as np
import pandas as pd

# Exercise 1 Data preparation
## Q1.1 Load the dataset, inspect feature names and target distribution. Comment on dataset imbalance

In [18]:
# Q1.1 Loading the dataset and counting how many samples are malignant or benign

df = pd.read_csv("../Data/data.csv")

y = df.diagnosis

unique, counts = np.unique(y, return_counts=True)
print(unique, counts, (counts / len(y))*100)

['B' 'M'] [357 212] [62.74165202 37.25834798]


The dataset consists of 30 numeric values of features that describe the characteristics of tumors computed from images of breasts. These include mean values, error estimates and worst-case values, and they have information on the tumor size, shape and texture etc.

The dataset also consists of 569 samples with 30 features each. The target variable is whether the tumor is malignant or benign, where 63% of the dataset is benign and 37% is malignant. This means there is a slight imbalance towards the benign class, which should be considered when evaluating model performance.